In [ ]:
import google.generativeai as genai
import pandas
import json

In [ ]:
import utils.gemini_functions as gemini_functions
import utils.functions as functions
gemini_functions.configurar_api()

In [ ]:
# Paso 1: Leer el archivo JSON de entrada
json_data = gemini_functions.read_json_file('scraped_texts.json')

In [ ]:
# Limpiar el JSON eliminando las claves con el valor "Error al scrapear"
json_limpio = functions.limpiar_json(json_data, "Error al scrapear")

total_claves = len(json_limpio.keys())  # Contar el número total de claves
print(f"Total de claves en el JSON: {total_claves}")

In [ ]:
# Fragmentar el JSON
fragmentos = functions.fragmentar_json(json_data, claves_por_fragmento=40)

# Guardar los fragmentos en la carpeta 'jsons'
functions.guardar_fragmentos(fragmentos, carpeta_destino="jsons")

In [ ]:
# Cargar el prompt desde un archivo
archivo_prompt = "prompts/analisis_de_conflicto.txt"
prompt = gemini_functions.cargar_prompt(archivo_prompt)

In [ ]:
# Procesar cada fragmento de JSON
for i, fragmento in enumerate(fragmentos, start=1):
    # Llamada a la función para enviar la solicitud a Gemini con el fragmento actual
    response = gemini_functions.enviar_a_gemini(prompt, fragmento)

    # Procesar la respuesta
    if response:
        print(f"Respuesta con JSON para conflicto {i}:", response)

        # Guardar la respuesta en un archivo de texto con nombre secuencial
        archivo_salida = f"respuesta_gemini_{i}.txt"
        gemini_functions.guardar_respuesta_en_txt(response, archivo_salida)
    else:
        print(f"No se obtuvo una respuesta válida para conflicto {i}.")